# 第二章：字符串和文本

## 2.1 使用多个界定符分割字符串

问题：需要将一个字符串分割为多个字段，但是分隔符（还有周围的空格）并不是固定的

解决方案：string对象的split方法只适应于非常简单的字符串分割情形，它并不允许有多个分隔符或者分隔符周围不确定的空格
，当你需要更加灵活的切割字符串的时候，最好使用re.split()方法

In [2]:
line = 'asdf fjdk;  afed, fjed,asdf,   foo'
import re
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo']

分隔符可以是逗号，分好或者空格，并且后面紧跟着任意个的空格

当你使用re.split函数时候，需要特别注意的是正则表达式中是否包含一个括号捕获分组，如果使用捕获分组，那么被匹配的文本也将出现结果列表中，比如

In [3]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjed', ',', 'asdf', ',', 'foo']

获取分割字符在某些情况下也是有用的，比如

In [4]:
values = fields[::2]
delimiters = fields[1::2] + ['']
values,delimiters

(['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo'],
 [' ', ';', ',', ',', ',', ''])

In [5]:
''.join(v+d for v, d in zip(values, delimiters))

'asdf fjdk;afed,fjed,asdf,foo'

如果你不想保留分割字符串到结果列表去，但仍然需要使用括号来分组正则表达式的话，确保你的分组是非捕获分组
形如，(?:......)，比如：

In [6]:
re.split(r'(?:,|;|\s)\s*', line) #这个跟In2的区别是？

['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo']

## 2.2 字符串开头或结尾匹配

问题：你需要指定的文本模式去检查字符串的开头或者结尾，比如，文件后缀，URL Scheme等

解决方案：简单方法是使用str.startswith() str.endswith()

In [1]:
filename = 'spam.txt'
filename.endswith('.txt')

True

In [2]:
url = 'http://www.python.org'
url.startswith('http:')

True

如果你想检查多种匹配可能，只需要讲所有的匹配项放到一个元组去，然后传递给startswith等

In [3]:
import os
filenames = os.listdir('.') # !
filenames

['.ipynb_checkpoints', 'CookBook_1.ipynb', 'CookBook_2.ipynb', 'somefile.txt']

In [4]:
[name for name in filenames if name.endswith(('ipynb','txt'))]

['CookBook_1.ipynb', 'CookBook_2.ipynb', 'somefile.txt']

In [5]:
any(name.endswith('.txt') for name in filenames)

True

In [6]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http', 'https', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()
choices = ['http:', 'ftp:']

In [7]:
url.startswith(choices)

TypeError: startswith first arg must be str or a tuple of str, not list

In [8]:
#类似的操作也可以使用切片来实现；还能使用正则表达式实现
#切片省略
#正则表达式

import re
url = 'http://www.python.org'
re.match('http:|https:|ftp:', url)

<re.Match object; span=(0, 5), match='http:'>

## 2.3 用shell通配符匹配字符串

问题：你想使用linux shell中的常用的通配符，比如，*.py,Dat[0-9]*.csv等去匹配文本字符串

解决方法：fnmatch模块提供两个函数 fnmatch和fnmatchcase，可以用来实现这样的匹配，比如

In [9]:
from fnmatch import fnmatch, fnmatchcase

fnmatch('foo.txt', '*.txt'), fnmatch('too.txt', '?oo.txt'),fnmatch('Dat45.csv', 'Dat[0-9]*')

(True, True, True)

In [10]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

fnmatch对底层操作系统大小写敏感，如果对这个区别很在意，可以使用fnmatchcase来代替，它完全使用你的模式大小写匹配，比如

In [14]:
fnmatch('foo.txt', '*.TXT')

True

In [15]:
fnmatchcase('foo.txt','*.TXT')

False

这两个函数通常会被忽略的一个特性是在处理非文件名的字符串时，它们是有用的，比如，一个街道
数据

In [16]:
address = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY'
]

In [17]:
from fnmatch import fnmatchcase

[addr for addr in address if fnmatchcase(addr, '* ST')]

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [18]:
[addr for addr in address if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

2.4 字符串匹配和搜索

问题：你想匹配或者搜索特定模式的文本

In [19]:
text = 'yeah, but no, but yeah, but no, but yeah'
text.find('no')

10

对于复杂的匹配需要使用正则表达式和re模块，为了解释正则表达式的基本原理，假设你想匹配数字
格式的日期字符串，比如11/22/2012

In [20]:
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

import re

if re.match(r'\d+/\d+/\d+',text1):
    print('yes')
else:
    print('no')

if re.match(r'\d+/\d+/\d+',text2):
    print('yes')
else:
    print('no')

yes
no


如果你想使用同一个模式做多次匹配，你应该先将模式字符串预编译为模式对象，比如

In [21]:
datepat = re.compile(r'\d+/\d+/\d+')

if datepat.match(text1):
    print('yes')
else:
    print('no')
    
if datepat.match(text2):
    print('yes')
else:
    print('no')

yes
no


match总是从字符串开始去匹配，如果你想查找字符串任意部分的模式出现位置，使用findall方法

In [22]:
text = 'Todau is 11/27/2012, PyCon starts 3/13/2013.'
datepat.findall(text)

['11/27/2012', '3/13/2013']

在定义正则式的时候，通常会利用括号去捕获分组，比如：

In [27]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

m = datepat.match('11/28/2012')
m,m.group(0), m.group(1), m.group(2),m.group(3), m.groups()

(<re.Match object; span=(0, 10), match='11/28/2012'>,
 '11/28/2012',
 '11',
 '28',
 '2012',
 ('11', '28', '2012'))

In [30]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [31]:
for month, day, year in datepat.findall(text):
    print('{}-{}-{}'.format(year, month, day))

2012-11-27
2013-3-13


findall方法会搜索文本并以列表形式返回所有的匹配，如果想以迭代凡是返回匹配，可以使用
finditer方法来代替，比如

In [32]:
for m in datepat.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


In [33]:
m = datepat.match('11/27/2012abcdefg')
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [34]:
m.group()

'11/27/2012'

In [35]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)$')
datepat.match('11/27/2012abcedfg')

In [36]:
datepat.match('11/27/2012')

<re.Match object; span=(0, 10), match='11/27/2012'>

## 2.5 字符串搜索和替换

问题：想在字符串中搜索和匹配指定的文本模式

解决方法：str.replace;re.sub()

In [38]:
text = 'yeah, but no, but yeah, but no,but yeah'
text.replace('yeah', 'yep')

'yep, but no, but yep, but no,but yep'

In [46]:
text = 'Today is 11/27/2012, PyCon starts 3/13/2013.'
import re
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-\2', text) #反斜杠数字比如\3指向前面模式的捕获组号

'Today is 2012-11-27, PyCon starts 2013-3-13.'

对于更复杂的替换，可以传递一个替换回调函数来代替，比如

In [49]:
from calendar import month_abbr

datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))

datepat.sub(change_date, text)


'Today is 27 Nov 2012, PyCon starts 13 Mar 2013.'

In [50]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
newtext, n

('Today is 2012-11-27, PyCon starts 2013-3-13.', 2)

## 2.6 字符串忽略大小写的搜索替换

问题：你需要以忽略大小写的方式搜索与替换文本字符串

解决方法：re模块时给这些才啊哦做提供re.IGNORECASE标志参数

## 2.7 最短匹配模式（非贪心）

问题：你正在试用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹配，而你想修改
它变成查找最短的可能匹配

解决方案：这个问题一般出现在需要匹配一对分隔符之间的文本的时候（比如引号包含的字符串）

In [52]:
str_pat = re.compile(r'\"(.*)\"') # 匹配被双引号包含的文本
text1 = 'Computer says "no."'
str_pat.findall(text1)

['no.']

In [54]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

正则表达式中*操作符是贪婪的，因为匹配操作会查找最长的可能匹配，第二个例子中返回结果
并不是我们想要的，为了修正这个问题，可以在模式中*操作符后面加上？修饰符

In [55]:
str_pat = re.compile(r'\"(.*?)\"')
str_pat.findall(text2)

['no.', 'yes.']

可以在*或者+这样的操作符后面添加一个？，可以强制匹配寻找最短的可能匹配

## 2.8 多行匹配模式

问题：你正试使用正则表达式去匹配一大块的文本，而你需要跨越多行去匹配

解决方案：这个问题很典型的出现当你用.去匹配任意字符的时候，忘记了.不能匹配换行符的事实
，比如，假设你想试着去匹配C语言分割的注释

In [57]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a 
multiline comment */'''
comment.findall(text1)

[' this is a comment ']

In [58]:
comment.findall(text2)

[]

In [59]:
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
comment.findall(text2)

[' this is a \nmultiline comment ']

(?:.|\n)指定了一个非捕获组，也就是定义了一个仅仅用来做匹配，而不能通过单独捕获或者编号的组

更为简单使用一个标志参数 re.DOTALL,他可以让正则表示中的点匹配包括换行符在内的任意字符

In [61]:
comment = re.compile(r'/\*(.*?)\*/',re.DOTALL)
comment.findall(text2)

[' this is a \nmultiline comment ']

### 2019.07.17

2.9 将Unicode文本标准化

问题：你正在处理Unicode字符串，需要确保所有字符串在底层有相同的表示

解决方案：在Unicode中，某些字符能够用多个合法的编码表示，为了说明，比如

In [2]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalape\u0303o'
s1, s2

('Spicy Jalapeño', 'Spicy Jalapẽo')

In [6]:
s1 == s2, len(s1), len(s2)

(True, 14, 14)

这里的文本Spicy Jalapeno使用了两种形式来表示，第一种使用整体字符（U+00F1），第二种使用拉丁字符n后面跟一个~的
组合字符（U+0303）,在需要比较字符串的程序中使用字符的多种表示会产生问题，为了修正这个问题，你可以使用unicodedata
模块先将文本标准化

In [8]:
import unicodedata 

t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)

t1 == t2

True

In [9]:
print(ascii(t1))

'Spicy Jalap\u1ebdo'


In [12]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
t3 == t4

True

In [13]:
print(ascii(t3))

'Spicy Jalape\u0303o'


normalize第一个参数指定字符串标准化的方式，NFC表示字符应该是整体组成，而NFD表示字符应该分解为多个组合字符表示

In [14]:
s = '\ufb01'
s

'ﬁ'

In [15]:
unicodedata.normalize('NFD', s), unicodedata.normalize('NFKD', s), unicodedata.normalize('NFKC', s)

('ﬁ', 'fi', 'fi')

讨论：标准化对于任何需要以一致的方式处理Unicode文本的程序都是非常重要的，当处理来自用户输入的字符串而你很难去
去控制编码的时候尤其如此
在清理和过滤文本的时候字符的标准化也是很重要的，比如，假设你想清除掉一些文本上面的变音符的时候

..................

## 2.11 删除字符串中不需要的字符

问题：你想去掉文本字符串开头、结尾或者中间不想要的字符，比如，空白

解决方案：strip方法能用删除开始或结尾的字符，lstrip或rstrip分别从左或从右执行删除操作，默认情况下，这些方法会去除空白
字符，但是可以指定其他字符，比如，

In [18]:
s = ' hello world  \n'
s.strip()

'hello world'

In [19]:
s.lstrip()

'hello world  \n'

In [20]:
s.rstrip() # !

' hello world'

In [21]:
print(s)

 hello world  



In [24]:
t = '----------hello-==========='
t.lstrip('-')

'hello-==========='

In [25]:
t.strip('-=')

'hello'

讨论：这些strip方法在读取和清理数据以备后续处理的时候会经常被用到的，比如，你可以用他们去掉空格，引号和其他任务，但是注意去除操作不会对字符串的中间的文本产生任何影响，比如

In [26]:
s = '    hello              world   \n'
s = s.strip()
s

'hello              world'

如果你想处理中间的空格，那么你需要其他的技术，比如，使用replace方法或者用正则表达式替换

In [27]:
s

'hello              world'

In [28]:
s.replace('   ','')

'hello  world'

In [30]:
import re 

re.sub('\s+', '', s)

'helloworld'

通常情况下你想将字符串strip操作和其他迭代操作相结合，比如从文件读取多行数据，如果是这样的话，那么生成器就可以大显身手；对于更高阶的strip，你可能需要使用translate方法

In [33]:
with open('somefile.txt') as f:
    i = 0
    lines = (line.strip() for line in f) 
    # 这种方式非常高效，因为它不需要预先读取所有数据到一个临时的列表中去，只是创建一个生成器
    for line in lines:
        print(line)
        i += 1
        if i > 5:
            break

=== Keeping the Last N Items

==== Problem

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.


## 2.12 审查清理文本字符串

问题：在你的网站页面表单中输入文本python，然后你想讲这些字符清理掉

解决方案：文本清理问题会涉及到包括文本解析与数据处理等一系列问题，简单情形下，会使用str.upper() str.lower() 将文本转为
标准格式；使用str.place或者re.sub的简单替换操作能删除或者改变指定的字符序列，或者使用2.9小节的unicodedata.normalize
函数将unicode文本标准化

然而有时候你想在清理操作上更进一步，比如，消除整个区间上的字符或者去除变音符，使用经常被忽视的str.translate方法
比如，

In [36]:
s = 'pýthöñ\fis\tawesome\r\n'
s

'pýthöñ\x0cis\tawesome\r\n'

第一步清理空白字符，为了这样做，先创建一个小的转换表然后使用translate方法

In [38]:
remap = {
    ord('\t') : " ", # \t 映射成空格
    ord('\f') : ' ', #  \f 映射成空格
    ord('\r') : None # \r直接被删除
}
a = s.translate(remap)
a

'pýthöñ is awesome\n'

删除所有的和音符

In [39]:
import unicodedata
import sys

cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))
# sys.maxunicode ？ unicodedata.combining ？
b = unicodedata.normalize('NFD', a) # ?
b, b.translate(cmb_chrs)

('pýthöñ is awesome\n', 'python is awesome\n')

dict.fromkeys方法构建一个字典，每个Unicode和音符作为键，对应的值全部为None\n
然后使用unicodedata.normalize()将原始输入标准化为分解形式字符，然后再调用translate函数删除所有重音符

ValueError: not a digit

In [52]:
digitmap = {c : ord('0') + unicodedata.digit(chr(c)) 
            for c in range(sys.maxunicode) 
            if unicodedata.category(chr(c)) == 'Nd'}
len(digitmap)

580

In [54]:
x = '\u0661\u0662\u0663'
x.translate(digitmap) #?

'123'

......................

## 2.13 字符串对齐

问题：你想通过某种方式来格式化字符串

解决方案：基本的字符串的对齐操作，可以使用字符串ljust(), rjust(), center()

In [57]:
text = 'Hello World'
text.ljust(20),text.rjust(20), text.center(20)

('Hello World         ', '         Hello World', '    Hello World     ')

所有这些方法都能接受一个可选的填充字符

In [58]:
text.rjust(20, '='), text.center(20, '*')

('=========Hello World', '****Hello World*****')

函数format同样可以用来很容易的对齐字符串，你要做的就是使用，或者 ^ 字符，后面紧跟一个指定的宽度，比如

In [59]:
format(text, '>20'), format(text, '<20'), format(text, '^20')

('         Hello World', 'Hello World         ', '    Hello World     ')

如果你想指定一个非空格的填充字符，将它写到对齐字符的前面即可

In [64]:
format(text, '=>20s'), format(text, '*^20s')

('=========Hello World', '****Hello World*****')

当格式化多个值的时候，这些格式化代码也可以被用在format方法中，比如

In [66]:
'{:>10s} {:>10s}'.format('Hello', 'World')

'     Hello      World'

format函数一个好处它不仅适用于字符串，它可以用来格式化任何值，使得它非常的通用，比如，格式化数字

In [68]:
x = 1.23345
format(x, '>10'), format(x, '^10.2f')

('   1.23345', '   1.23   ')

讨论：在老的代码中，你经常会看到被用来格式化文本的%操作符，比如，

In [69]:
'%-20s' % text, '%20s' % text

('Hello World         ', '         Hello World')

优先选择format函数或者方法，format 要比 % 操作符的功能更为强大，并且format比使用ljust，rjust，center方法更为通用
，因为它可以用来格式化任意对象，而不是仅仅是字符串

## 2.14 合并拼接字符串

In [70]:
parts = ['Is', 'Chicago', 'Not', 'Chicago?']
' '.join(parts), ','.join(parts), ''.join(parts)

('Is Chicago Not Chicago?', 'Is,Chicago,Not,Chicago?', 'IsChicagoNotChicago?')

In [72]:
a = 'Is Chicago'
b = 'Not Chicago'
a + '   ' + b

'Is Chicago   Not Chicago'

In [73]:
print('{} {}'.format(a, b)), print(a + '   ' + b )

Is Chicago Not Chicago
Is Chicago   Not Chicago


(None, None)

In [74]:
a = 'Hello' 'World'
a

'HelloWorld'

In [75]:
#性能上 join合并更胜一筹
data = ['ACME', 50, 91.1]
','.join(str(d) for d in data)

'ACME,50,91.1'

In [77]:
#不必要的字符串连接操作
a = 'liugang'
b = '13810330623'
c = 'age=31'
print(a + ':' + b + ':' + c) # Ugly
print(':'.join([a, b, c])) # still ugly
print(a, b, c, sep=':') # Better

liugang:13810330623:age=31
liugang:13810330623:age=31
liugang:13810330623:age=31


In [78]:
#Version 1
#f.write(chunk1+chunk2)

#Version 2
#f.write(chunk1)
#f.write(chunk2)

#如果两个字符串很小，那么第一个版本性能会更好些，因为I/O系统调用天生就慢
#另一方面，如果两个字符串很大，那么第二个版本可能会更加高效，因为它避免创建一个很大的临时结果并且复制
#大量数据，还是那句话，有时候需要根据你的程序特点，来决定使用哪种方案

In [82]:
#如果你准备编写构建大量小字符串的输出代码，最好考虑使用生成器函数，利用yield语句产生输出片段
def sample():
    yield 'Is'
    yield 'ChildProcessError'
    yield 'Not'
    yield 'Chicago?'
    
text = ''.join(sample())
text
#或者你也可以将字符串片段重定向到I/O
#for part in sample():
#   f.write(part)a

'IsChildProcessErrorNotChicago?'

In [83]:
#再或者还可以写出一个结合I/O操作的混合方案
def combine(source, maxsize):
    parts = []
    size = 0
    for part in source:
        parts.append(part)
        size += len(part)
        if size > maxsize:
            yield ''.join(parts)
            parts = []
            size = 0
        yield ''.join(parts)
        
#结合文件操作
with open('215.txt', 'w') as f:
    for part in combine(sample(), 32768):
        f.write(part)

#### 2019.07.19

## 2.15 字符串中插入变量

问题：你想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉

解决方案：python并没有在字符串中简单替换变量值提供的支持，但是通过字符串的format方法来解决这个问题，比如

In [1]:
s = '{name} has {n} message.'
s.format(name='Guido',n=37)

'Guido has 37 message.'

或者，如果要被替换的变量能在变量域中找到，那么你可以结合使用format map和vars

In [3]:
name = 'Guido'
n = 37 
s.format_map(vars())

'Guido has 37 message.'

vars还有一个有意思的特性就是它也适用于对象实例，比如；

In [4]:
class Info:
    def __init__(self, name, n):
        self.name = name
        self.n = n 

a = Info('Guido', 37)
s.format_map(vars(a))

'Guido has 37 message.'

format和format map一个缺陷是他们不能很好的处理变量缺失的情况

In [5]:
s.format(name='Gfuidl')

KeyError: 'n'

一种避免这种错误的方法是另外定义一个含有__missing__方法的字典对象

In [6]:
class safesub(dict): # 没看懂
    def __miss__(self, key):
        return '{' + key + '}'

#__missing__方法可以让你定义如何处理缺失的值，类中，这个方法被定为缺失的值返回一个占位符，你可以发现缺失的值
#出现在结果字符串中，而不是产生一个keyerror异常

In [7]:
del n

In [9]:
s.format_map(safesub(vars())) # 与书中执行结果不一致

KeyError: 'n'

如果你发现自己在代码中频繁的执行这些步骤，你可以将变量替换步骤用一个工具函数封装起来

In [20]:
import sys

def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals)) # ?

#使用sys._getframe(1)返回调用者的栈帧，可以从中访问属性f locals来获得局部变量，大部分情况在代码中去操作栈帧是
#不推荐的，但是对于字符串工具函数而言是非常有用的，另外值得注意的是f locals是一个复制调用函数的本地变量的字典
#尽管你可以改变 f locals的内容，但是这个修改对于后面的变量访问是没有任何影响的

In [11]:
name = 'Guido'
n = 37
print(sub('Hello  {name}'))
print(sub('You have {n} message'))
print(sub('Your favorite color is {color}'))

Hello  Guido
You have 37 message


KeyError: 'color'

另一个解决方案：字符串格式化代码

In [18]:
name = 'Guido'
n = 37
'%(name) has %(n) message.' % vars() # ?

ValueError: unsupported format character 'm' (0x6d) at index 17

In [19]:
import string

s = string.Template('$name has $n message.')
s.substitute(vars())

'Guido has 37 message.'

## 2.16 以指定列宽格式化字符串 

问题：你有一些长字符串，想以指定的列宽将他们重新格式化

解决方案：使用textwrap模块来格式化字符串的输出，比如

In [24]:
s = "Look into my eyes,look into my eyes, theeyes, the eyes, the eyes, \
not around theeyes, don't look around the eyes,  \
ookinto myeyes, you're under."
s

"Look into my eyes,look into my eyes, theeyes, the eyes, the eyes, not around theeyes, don't look around the eyes,  ookinto myeyes, you're under."

In [25]:
import textwrap

print(textwrap.fill(s, 70))

Look into my eyes,look into my eyes, theeyes, the eyes, the eyes, not
around theeyes, don't look around the eyes,  ookinto myeyes, you're
under.


In [26]:
print(textwrap.fill(s,45))

Look into my eyes,look into my eyes, theeyes,
the eyes, the eyes, not around theeyes, don't
look around the eyes,  ookinto myeyes, you're
under.


In [27]:
print(textwrap.fill(s, 40, initial_indent='     '))

     Look into my eyes,look into my
eyes, theeyes, the eyes, the eyes, not
around theeyes, don't look around the
eyes,  ookinto myeyes, you're under.


In [28]:
print(textwrap.fill(s, 40, subsequent_indent='      '))

Look into my eyes,look into my eyes,
      theeyes, the eyes, the eyes, not
      around theeyes, don't look around
      the eyes,  ookinto myeyes, you're
      under.


textwrap模块对于字符串打印是非常有用的，特别是当你希望输出自动匹配终端大小的时候，你可以使用os.get_terminal_size()
方法来获取终端的大小尺寸

In [29]:
import os

os.get_terminal_size().columns

120

## 2.17 在字符串中处理html和xml

问题：你想讲HTML或者XML实例如&entity或者&#code，替换为对应的文本，再者，你需要转换文本中特定的字符，比如，< > & 

解决方案：如果你想替换问题字符串中的< > ，使用html.escape()函数可以很容易完成，比如

In [30]:
s = 'Elements are written as "<tag>text</tag>".'

import html

print(s)

Elements are written as "<tag>text</tag>".


In [31]:
print(html.escape(s))

Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


In [32]:
print(html.escape(s, quote=False))

Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


如果你正在处理的是ASCII文本，并且向将非ASCII文本对应的编码实体嵌入进入，可以给某些I/O函数传递参数
errors='xmlcharrefreplace'来达到这个目的，比如

In [33]:
s= 'Spicy Jalapeño'
s.encode('ascii', errors = 'xmlcharrefreplace')

b'Spicy Jalape&#241;o'

....................................

## 2.18 字符串令牌解析

问题：你有字符串，想从左至右将其解析为一个字符串

解决方案：假如你有下面这样文本字符串

In [ ]:
text = 'foo = 23+42*10'